In [4]:
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

file = glob.glob("train/train.tsv")
pd.read_csv("train/train.tsv", delimiter='\t')
data_list = []
data_list.append(pd.read_csv("train/train.tsv", delimiter='\t', index_col=0))
df = pd.concat(data_list)
#df.info()

In [13]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error as mae
df_train, df_val = train_test_split(df, test_size=0.2)
print(type(df))
print(type(df_train))
print(type(df_val))

col = "Vertical_Distance_To_Hydrology"

train_y = df_train[col]
train_x = df_train.drop(col, axis=1)

val_y = df_val[col]
val_x = df_val.drop(col, axis=1)

trains = lgb.Dataset(train_x, train_y)
valids = lgb.Dataset(val_x, val_y)

params = {
    "objective": "regression",
    "metrics": "mae"
}

model = lgb.train(params, trains, valid_sets=valids,
                  num_boost_round=1000,
                  callbacks=[lgb.early_stopping(stopping_rounds=10, verbose=True)])


<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001495 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2015
[LightGBM] [Info] Number of data points in the train set: 232404, number of used features: 53
[LightGBM] [Info] Start training from score 46.323385
Training until validation scores don't improve for 10 rounds
Did not meet early stopping. Best iteration is:
[1000]	valid_0's l1: 15.3393


In [16]:
vals = model.predict(val_x)
mae(vals, val_y)

15.339275512524157

In [17]:
df_test = pd.read_csv("train/test.tsv", delimiter='\t', index_col=0)
predict = model.predict(df_test)

df_test["Vertical_Distance_To_Hydrology"] = predict
df_test[["Vertical_Distance_To_Hydrology"]].to_csv("submit_test.csv")